# DSCI478 Kaggle Project - Credit Card Fraud Detection
### Nick Brady, Jakob Wickham

Note for us:
- If you want the PDF to not display a cell, click the three dots on the cell, click "Add Cell Tag", and put "remove_cell"
- If you want to hide the code instead, put "remove_input"

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# Figure out which one to use
# import torch as t
# import tensorflow as tf

In [14]:
df = pd.read_csv("combined_data.csv")

In [15]:
df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0
...,...,...,...,...,...,...,...,...,...
1754150,1754150,2018-09-30 23:56:36,161,655,54.24,15810996,182,0,0
1754151,1754151,2018-09-30 23:57:38,4342,6181,1.23,15811058,182,0,0
1754152,1754152,2018-09-30 23:58:21,618,1502,6.62,15811101,182,0,0
1754153,1754153,2018-09-30 23:59:52,4056,3067,55.40,15811192,182,0,0


# Introduction

*Talk about the importance of fraud detection and fraud prevention*

*State the dataset was synthetically generated*

# Data Cleaning and Preprocessing

In [ ]:
# Load and Preprocess Data

df.columns = df.columns.str.strip().str.upper()

# Drop unnecessary column
df.drop(columns=['TX_FRAUD_SCENARIO'], inplace=True)

# Convert TX_DATETIME and extract key time features
df['TX_DATETIME'] = pd.to_datetime(df['TX_DATETIME'])
df['TX_HOUR'] = df['TX_DATETIME'].dt.hour
df['TX_DAYOFWEEK'] = df['TX_DATETIME'].dt.dayofweek
df.drop(columns=['TX_DATETIME'], inplace=True)


# Time-Based Split
split_point = df['TX_TIME_DAYS'].quantile(0.8)
train_df = df[df['TX_TIME_DAYS'] < split_point].copy()
test_df = df[df['TX_TIME_DAYS'] >= split_point].copy()

*Dropping the TX_FRAUD_SCENARIO column and turning the TX_DATETIME column to a proper DateTime*

# Exploratory Data Analysis (EDA)

*General stuff, like fraud percentage, rolling window sample distributions, etc*

*Graphs such as ratio of fraud within sliding windows, and then some*

# Feature Engineering

*Imbalanced techniques, adding columns for rolling windows and sampling distribution z-scores, etc*

In [ ]:
# Feature Engineering Rolling Windows with decay

scaler = StandardScaler()
train_df['TX_AMOUNT_Z_SCORE'] = scaler.fit_transform(train_df[['TX_AMOUNT']])
test_df['TX_AMOUNT_Z_SCORE'] = scaler.transform(test_df[['TX_AMOUNT']])

train_df['TX_AMOUNT_PERCENTILE'] = train_df['TX_AMOUNT'].rank(pct=True)
test_df['TX_AMOUNT_PERCENTILE'] = test_df['TX_AMOUNT'].rank(pct=True)


# Optimize Decay Factor 

decay_values = np.linspace(0.8, 0.99, 20)  
best_decay, best_corr = 0.95, float('-inf')  

for decay in decay_values:
    temp_df = train_df.copy()
    temp_df['TERMINAL_FRAUD_RATIO_28D'] = (
        temp_df.groupby('TERMINAL_ID')['TX_FRAUD']
        .shift(1)
        .ewm(alpha=1-decay)
        .mean()
    )
    correlation = temp_df['TERMINAL_FRAUD_RATIO_28D'].corr(temp_df['TX_FRAUD'])
    
    if correlation > best_corr:
        best_corr = correlation
        best_decay = decay

optimal_decay = best_decay  # Best decay factor based on correlation

# Apply the optimized decay factor
train_df['TERMINAL_FRAUD_RATIO_28D'] = (
    train_df.groupby('TERMINAL_ID')['TX_FRAUD']
    .shift(1)
    .ewm(alpha=1-optimal_decay)
    .mean()
)
test_df['TERMINAL_FRAUD_RATIO_28D'] = (
    test_df.groupby('TERMINAL_ID')['TX_FRAUD']
    .shift(1)
    .ewm(alpha=1-optimal_decay)
    .mean()
)

# Terminal Fraud Ratios
train_df['TERMINAL_FRAUD_RATIO_7D'] = (
    train_df.groupby('TERMINAL_ID')['TX_FRAUD']
    .shift(1)
    .rolling(7, min_periods=1)
    .mean()
)
test_df['TERMINAL_FRAUD_RATIO_7D'] = (
    test_df.groupby('TERMINAL_ID')['TX_FRAUD']
    .shift(1)
    .rolling(7, min_periods=1)
    .mean()
)

# Customer Spending Trends
train_df['SPENDING_RATIO_CHANGE'] = (
    train_df.groupby('CUSTOMER_ID')['TX_AMOUNT']
    .transform(lambda x: x.pct_change().rolling(14, min_periods=1).mean())
)
test_df['SPENDING_RATIO_CHANGE'] = (
    test_df.groupby('CUSTOMER_ID')['TX_AMOUNT']
    .transform(lambda x: x.pct_change().rolling(14, min_periods=1).mean())
)

train_df['SPENDING_Z_SCORE_28D'] = (
    train_df.groupby('CUSTOMER_ID')['TX_AMOUNT']
    .transform(lambda x: (x - x.mean()) / (x.std() + 1))
)
test_df['SPENDING_Z_SCORE_28D'] = (
    test_df.groupby('CUSTOMER_ID')['TX_AMOUNT']
    .transform(lambda x: (x - x.mean()) / (x.std() + 1))
)


# Anomaly Detection (Local Outlier Factor)

features_for_lof = ['TX_AMOUNT_Z_SCORE', 'SPENDING_RATIO_CHANGE', 'TERMINAL_FRAUD_RATIO_28D']
train_df[features_for_lof] = train_df[features_for_lof].fillna(train_df[features_for_lof].median())
test_df[features_for_lof] = test_df[features_for_lof].fillna(test_df[features_for_lof].median())

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.005, novelty=True)
lof.fit(train_df[features_for_lof].values)

train_df['ANOMALY_SCORE'] = lof.decision_function(train_df[features_for_lof].values)
test_df['ANOMALY_SCORE'] = lof.decision_function(test_df[features_for_lof].values)

# Normalize Anomaly Scores
scaler = MinMaxScaler()
train_df['ANOMALY_SCORE'] = scaler.fit_transform(train_df[['ANOMALY_SCORE']])
test_df['ANOMALY_SCORE'] = scaler.transform(test_df[['ANOMALY_SCORE']])


# Model Selection

*Isolation Forest, Balanced Random Forest, etc*

# Model Training and Evaluation

# Model Interpretation and Explainability

*Graphs such as feature importance and within-feature / within-fraud boxplots*

# Conclusion

*Talk about the three scenarios that we used to make a transaction fraud*

# References

- Reproducible Machine Learning for Credit Card Fraud Detection - Practical Handbook
    - Yann-Aël Le Borgne, Wissam Siblini, Bertrand Lebichot, and Gianluca Bontempi
    - https://github.com/Fraud-Detection-Handbook/fraud-detection-handbook